# Data Resampling - Daily

In [1]:
import dai
import pandas as pd
from datetime import datetime, timedelta
from bigcharts import Kline

In [2]:
sd = '2020-01-01 00:00:00.000'
ed = '2026-02-13 23:59:59.999'

In [3]:
def get_data_d(sql, sd, ed, freq, instruments=[]):

    date_obj = datetime.strptime(sd, '%Y-%m-%d %H:%M:%S.%f')
    
    if freq == "week":
        start_of_week = date_obj - timedelta(days=date_obj.weekday())
        sd = start_of_week.strftime('%Y-%m-%d 00:00:00.000')
    
    elif freq == "fortnight":
        days_since_year_start = (date_obj - datetime(date_obj.year, 1, 1)).days
        fortnight_start = days_since_year_start - (days_since_year_start % 14)
        sd = (datetime(date_obj.year, 1, 1) + timedelta(days=fortnight_start)).strftime('%Y-%m-%d 00:00:00.000')
    
    elif freq == "month":
        start_of_month = datetime(year=date_obj.year, month=date_obj.month, day=1)
        sd = start_of_month.strftime('%Y-%m-%d 00:00:00.000')
    
    elif freq == "bimonthly":
        bimonthly_start_month = (date_obj.month - 1) // 2 * 2 + 1
        start_of_bimonthly = datetime(year=date_obj.year, month=bimonthly_start_month, day=1)
        sd = start_of_bimonthly.strftime('%Y-%m-%d 00:00:00.000')
    
    elif freq == "quarter":
        quarter_start_month = 3 * ((date_obj.month - 1) // 3) + 1
        start_of_quarter = datetime(year=date_obj.year, month=quarter_start_month, day=1)
        sd = start_of_quarter.strftime('%Y-%m-%d 00:00:00.000')
    
    elif freq == "quadmonthly":
        quadmonthly_start_month = 4 * ((date_obj.month - 1) // 4) + 1
        start_of_quadmonthly = datetime(year=date_obj.year, month=quadmonthly_start_month, day=1)
        sd = start_of_quadmonthly.strftime('%Y-%m-%d 00:00:00.000')
    
    elif freq == "semiannual":
        semiannual_start_month = 1 if date_obj.month <= 6 else 7
        start_of_semiannual = datetime(year=date_obj.year, month=semiannual_start_month, day=1)
        sd = start_of_semiannual.strftime('%Y-%m-%d 00:00:00.000')
    
    elif freq == "year":
        sd = datetime(year=date_obj.year, month=1, day=1).strftime('%Y-%m-%d 00:00:00.000')

    else:
        sd = sd

    sql_all = f"""
    WITH
    data_all AS (
        {sql}
    )
    SELECT *
    FROM data_all
    """

    if instruments == []:
        return dai.query(sql_all, filters={'date':[sd, ed]}).df()
    else:
        return dai.query(sql_all, filters={'date':[sd, ed], 'instrument':instruments}).df()

## Day

In [4]:
sql_day = """
SELECT
    date,
    instrument,
    open,
    close,
    high,
    low,
    volume,
    amount,
    turn,
    deal_number,
FROM cn_stock_bar1d
ORDER BY date
"""

df_day = get_data_d(sql_day, sd, ed, freq="day", instruments=['600519.SH'])
df_day

,date,instrument,open,close,high,low,volume,amount,turn,deal_number
0,2020-01-02,600519.SH,8255.352493,8269.989643,8380.207381,8167.529594,14809916,1.669684e+10,0.011789,75243
1,2020-01-03,600519.SH,8174.848169,7893.522150,8174.848169,7881.373316,13031878,1.426638e+10,0.010374,76373
2,2020-01-06,600519.SH,7837.169123,7889.350562,7998.470514,7811.114997,6341478,6.853918e+09,0.005048,34135
3,2020-01-07,600519.SH,7885.764461,8010.399791,8043.113821,7877.714028,4785359,5.220697e+09,0.003809,28504
4,2020-01-08,600519.SH,7941.019701,7963.634098,8017.498809,7922.942821,2500825,2.720372e+09,0.001991,17510
...,...,...,...,...,...,...,...,...,...,...
1480,2026-02-09,600519.SH,12822.533806,12881.324870,13032.694966,12784.437872,5116846,7.809068e+09,0.004086,36705
1481,2026-02-10,600519.SH,12881.409340,12711.033512,12881.409340,12640.923479,3957596,5.953269e+09,0.003160,28965
1482,2026-02-11,600519.SH,12711.033512,12707.063426,12788.745838,12636.699983,3092846,4.648360e+09,0.002470,23410
1483,2026-02-12,600519.SH,12672.177349,12557.298258,12694.308468,12445.460084,4624582,6.874370e+09,0.003693,33714


In [5]:
Kline(df_day)

## Week

In [6]:
sql_week = """
WITH 
data_orgn AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
        turn,
        volume / turn AS turn_base,
        deal_number,
    FROM cn_stock_bar1d
),
data_freq AS (
    WITH 
    data1 AS (
        SELECT
            instrument,
            CONCAT(CAST(year(date) AS CHAR), CAST(week(date) AS CHAR)) AS period,
            FIRST_VALUE(date) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS date,
            FIRST_VALUE(open) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
            LAST_VALUE(close) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
            MAX(high)         OVER (PARTITION BY period, instrument) AS high,
            MIN(low)          OVER (PARTITION BY period, instrument) AS low,
            SUM(amount)       OVER (PARTITION BY period, instrument) AS amount,
            SUM(volume)       OVER (PARTITION BY period, instrument) AS volume,
            AVG(turn_base)    OVER (PARTITION BY period, instrument) AS turn_base,
            SUM(deal_number)  OVER (PARTITION BY period, instrument) AS deal_number,
            ROW_NUMBER()      OVER (PARTITION BY period, instrument ORDER BY date DESC) AS rn
        FROM data_orgn
    ),
    data2 AS (
        SELECT
            date,
            instrument,
            open,
            close,
            high,
            low,
            amount,
            volume,
            turn_base,
            deal_number,
        FROM data1
        WHERE rn = 1 
    )
    SELECT 
        date,
        instrument,
        open,
        close,
        high,
        low,
        amount,
        volume,
        volume / turn_base AS turn,
        deal_number,
    FROM data2
)
SELECT
    date,
    instrument,
    open,
    close,
    high,
    low,
    amount,
    volume,
    turn,
    deal_number,
FROM data_freq
ORDER BY date, instrument
"""

df_week = get_data_d(sql_week, sd, ed, freq="week", instruments=['600519.SH'])
df_week

,date,instrument,open,close,high,low,amount,volume,turn,deal_number
0,2019-12-30,600519.SH,8564.196354,8657.874113,8749.356299,8564.196354,7.494387e+09,6334947,0.005043,38984
1,2020-01-02,600519.SH,8255.352493,7893.522150,8380.207381,7881.373316,3.096322e+10,27841794,0.022164,151616
2,2020-01-06,600519.SH,7837.169123,8141.914582,8167.456409,7811.114997,2.289939e+10,20965836,0.016690,124459
3,2020-01-13,600519.SH,8141.914582,8105.321708,8264.134783,8057.824157,1.630883e+10,14635391,0.011651,91317
4,2020-01-20,600519.SH,8137.230694,7704.995660,8137.230694,7589.362177,1.691491e+10,15793382,0.012572,98018
...,...,...,...,...,...,...,...,...,...,...
309,2026-01-12,600519.SH,11987.126300,11673.742898,12087.645505,11656.848914,3.131811e+10,22303711,0.017811,165927
310,2026-01-19,600519.SH,11672.898199,11293.628260,11720.116884,11255.363386,3.589528e+10,26424687,0.021101,206122
311,2026-01-26,600519.SH,11323.277202,11834.235746,12205.903392,11167.007850,6.707268e+10,48881417,0.039034,370358
312,2026-02-02,600519.SH,12036.963553,12797.277300,13244.883404,11923.942800,6.758877e+10,45175598,0.036075,318234


In [7]:
Kline(df_week)

## Fortnight

In [8]:
sql_fortnight = """
WITH 
data_orgn AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
        turn,
        volume / turn AS turn_base,
        deal_number,
    FROM cn_stock_bar1d
),
data_freq AS (
    WITH 
    data1 AS (
        SELECT
            instrument,
            CONCAT(CAST(year(date) AS CHAR), CAST(FLOOR((week(date)-1) / 2) AS CHAR)) AS period,
            FIRST_VALUE(date) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS date,
            FIRST_VALUE(open) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
            LAST_VALUE(close) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
            MAX(high)         OVER (PARTITION BY period, instrument) AS high,
            MIN(low)          OVER (PARTITION BY period, instrument) AS low,
            SUM(amount)       OVER (PARTITION BY period, instrument) AS amount,
            SUM(volume)       OVER (PARTITION BY period, instrument) AS volume,
            AVG(turn_base)    OVER (PARTITION BY period, instrument) AS turn_base,
            SUM(deal_number)  OVER (PARTITION BY period, instrument) AS deal_number,
            ROW_NUMBER()      OVER (PARTITION BY period, instrument ORDER BY date DESC) AS rn
        FROM data_orgn
    ),
    data2 AS (
        SELECT
            date,
            instrument,
            open,
            close,
            high,
            low,
            amount,
            volume,
            turn_base,
            deal_number,
        FROM data1
        WHERE rn = 1 
    )
    SELECT 
        date,
        instrument,
        open,
        close,
        high,
        low,
        amount,
        volume,
        volume / turn_base AS turn,
        deal_number,
    FROM data2
)
SELECT
    date,
    instrument,
    open,
    close,
    high,
    low,
    amount,
    volume,
    turn,
    deal_number,
FROM data_freq
ORDER BY date, instrument
"""

df_fortnight = get_data_d(sql_fortnight, sd, ed, freq="fortnight", instruments=['600519.SH'])
df_fortnight

,date,instrument,open,close,high,low,amount,volume,turn,deal_number
0,2020-01-02,600519.SH,8255.352493,8141.914582,8380.207381,7811.114997,5.386261e+10,48807630,0.038853,276075
1,2020-01-13,600519.SH,8141.914582,7704.995660,8264.134783,7589.362177,3.322374e+10,30428773,0.024223,189335
2,2020-02-03,600519.SH,7208.796282,7874.786598,7882.105173,7172.203407,3.212196e+10,31193522,0.024832,169463
3,2020-02-10,600519.SH,7772.326550,8144.695641,8226.078194,7737.197390,3.575924e+10,32670909,0.026008,192227
4,2020-02-24,600519.SH,8090.684558,8456.613303,8606.644089,7684.284093,4.966309e+10,44981544,0.035808,255735
...,...,...,...,...,...,...,...,...,...,...
156,2025-12-15,600519.SH,11956.600333,11945.144750,11990.336157,11752.366626,3.463163e+10,24435193,0.019513,192470
157,2026-01-05,600519.SH,11699.083874,11987.126300,12138.074047,11699.083874,2.826384e+10,19920168,0.015907,151749
158,2026-01-12,600519.SH,11987.126300,11293.628260,12087.645505,11255.363386,6.721339e+10,48728398,0.038912,372049
159,2026-01-26,600519.SH,11323.277202,12797.277300,13244.883404,11167.007850,1.346614e+11,94057015,0.075109,688592


In [9]:
Kline(df_fortnight)

## Month

In [10]:
sql_month = """
WITH 
data_orgn AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
        turn,
        volume / turn AS turn_base,
        deal_number,
    FROM cn_stock_bar1d
),
data_freq AS (
    WITH 
    data1 AS (
        SELECT
            instrument,
            CONCAT(CAST(year(date) AS CHAR), CAST(month(date) AS CHAR)) AS period,
            FIRST_VALUE(date) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS date,
            FIRST_VALUE(open) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
            LAST_VALUE(close) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
            MAX(high)         OVER (PARTITION BY period, instrument) AS high,
            MIN(low)          OVER (PARTITION BY period, instrument) AS low,
            SUM(amount)       OVER (PARTITION BY period, instrument) AS amount,
            SUM(volume)       OVER (PARTITION BY period, instrument) AS volume,
            AVG(turn_base)    OVER (PARTITION BY period, instrument) AS turn_base,
            SUM(deal_number)  OVER (PARTITION BY period, instrument) AS deal_number,
            ROW_NUMBER()      OVER (PARTITION BY period, instrument ORDER BY date DESC) AS rn
        FROM data_orgn
    ),
    data2 AS (
        SELECT
            date,
            instrument,
            open,
            close,
            high,
            low,
            amount,
            volume,
            turn_base,
            deal_number,
        FROM data1
        WHERE rn = 1 
    )
    SELECT 
        date,
        instrument,
        open,
        close,
        high,
        low,
        amount,
        volume,
        volume / turn_base AS turn,
        deal_number,
    FROM data2
)
SELECT
    date,
    instrument,
    open,
    close,
    high,
    low,
    amount,
    volume,
    turn,
    deal_number,
FROM data_freq
ORDER BY date, instrument
"""

df_month = get_data_d(sql_month, sd, ed, freq="month", instruments=['600519.SH'])
df_month

,date,instrument,open,close,high,low,amount,volume,turn,deal_number
0,2020-01-02,600519.SH,8255.352493,7704.995660,8380.207381,7589.362177,8.708635e+10,79236403,0.063076,465410
1,2020-02-03,600519.SH,7208.796282,7735.733675,8226.078194,7172.203407,8.996048e+10,84368612,0.067162,474426
2,2020-03-02,600519.SH,7685.308694,8130.936720,8606.644089,7026.563767,1.229550e+11,113791314,0.090584,650710
3,2020-04-01,600519.SH,8174.848169,9263.120258,9513.708263,8072.388121,7.418191e+10,61863828,0.049247,376922
4,2020-05-06,600519.SH,9155.537207,10001.564466,10040.792027,9148.218632,6.385191e+10,47877941,0.038113,305135
...,...,...,...,...,...,...,...,...,...,...
69,2025-10-09,600519.SH,11926.783374,11877.033073,12358.672466,11753.363293,9.380856e+10,64871148,0.051803,492844
70,2025-11-03,600519.SH,11885.255577,12047.213986,12342.642736,11793.977479,8.804849e+10,60575962,0.048373,464358
71,2025-12-01,600519.SH,12051.366766,11633.028397,12144.970421,11488.083724,9.035978e+10,63856302,0.050992,505683
72,2026-01-05,600519.SH,11699.083874,11834.235746,12205.903392,11167.007850,1.625499e+11,117529983,0.093854,894156


In [11]:
Kline(df_month)

## Bimonth

In [12]:
sql_bimonth = """
WITH 
data_orgn AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
        turn,
        volume / turn AS turn_base,
        deal_number,
    FROM cn_stock_bar1d
),
data_freq AS (
    WITH 
    data1 AS (
        SELECT
            instrument,
            CONCAT(CAST(year(date) AS CHAR), CAST(FLOOR((month(date)-1) / 2) AS CHAR)) AS period,
            FIRST_VALUE(date) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS date,
            FIRST_VALUE(open) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
            LAST_VALUE(close) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
            MAX(high)         OVER (PARTITION BY period, instrument) AS high,
            MIN(low)          OVER (PARTITION BY period, instrument) AS low,
            SUM(amount)       OVER (PARTITION BY period, instrument) AS amount,
            SUM(volume)       OVER (PARTITION BY period, instrument) AS volume,
            AVG(turn_base)    OVER (PARTITION BY period, instrument) AS turn_base,
            SUM(deal_number)  OVER (PARTITION BY period, instrument) AS deal_number,
            ROW_NUMBER()      OVER (PARTITION BY period, instrument ORDER BY date DESC) AS rn
        FROM data_orgn
    ),
    data2 AS (
        SELECT
            date,
            instrument,
            open,
            close,
            high,
            low,
            amount,
            volume,
            turn_base,
            deal_number,
        FROM data1
        WHERE rn = 1 
    )
    SELECT 
        date,
        instrument,
        open,
        close,
        high,
        low,
        amount,
        volume,
        volume / turn_base AS turn,
        deal_number,
    FROM data2
)
SELECT
    date,
    instrument,
    open,
    close,
    high,
    low,
    amount,
    volume,
    turn,
    deal_number,
FROM data_freq
ORDER BY date, instrument
"""

df_bimonth = get_data_d(sql_bimonth, sd, ed, freq="bimonth", instruments=['600519.SH'])
df_bimonth

,date,instrument,open,close,high,low,amount,volume,turn,deal_number
0,2020-01-02,600519.SH,8255.352493,7735.733675,8380.207381,7172.203407,1.770468e+11,163605015,0.130238,939836
1,2020-03-02,600519.SH,7685.308694,9263.120258,9513.708263,7026.563767,1.971369e+11,175655142,0.139831,1027632
2,2020-05-06,600519.SH,9155.537207,10831.220435,10876.385072,9148.218632,1.393701e+11,101000419,0.080402,651508
3,2020-07-01,600519.SH,10869.129114,13227.315506,13445.734653,10839.661039,3.014573e+11,180691494,0.143840,1121247
4,2020-09-01,600519.SH,13268.037720,12364.893054,13534.583120,11846.462250,1.928262e+11,113308907,0.090200,736104
5,2020-11-02,600519.SH,12401.765168,14793.269734,14800.525693,12379.553051,2.569005e+11,143279965,0.114059,884454
6,2021-01-04,600519.SH,14807.929731,15717.145709,19456.925761,14678.210970,3.636830e+11,165553251,0.131789,1077415
7,2021-03-01,600519.SH,16133.400776,14858.277196,16140.804815,14069.006649,3.701823e+11,181494122,0.144479,1240002
8,2021-05-06,600519.SH,14808.077812,15371.263822,17177.370262,13816.010639,2.762600e+11,131406144,0.104606,917687
9,2021-07-01,600519.SH,15395.702988,11644.104164,15769.614754,11401.207254,3.923210e+11,223135195,0.177627,1559928


In [13]:
Kline(df_bimonth)

## Quarter

In [14]:
sql_quarter = """
WITH 
data_orgn AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
        turn,
        volume / turn AS turn_base,
        deal_number,
    FROM cn_stock_bar1d
),
data_freq AS (
    WITH 
    data1 AS (
        SELECT
            instrument,
            CONCAT(CAST(year(date) AS CHAR), CAST(FLOOR((month(date)-1) / 3) AS CHAR)) AS period,
            FIRST_VALUE(date) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS date,
            FIRST_VALUE(open) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
            LAST_VALUE(close) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
            MAX(high)         OVER (PARTITION BY period, instrument) AS high,
            MIN(low)          OVER (PARTITION BY period, instrument) AS low,
            SUM(amount)       OVER (PARTITION BY period, instrument) AS amount,
            SUM(volume)       OVER (PARTITION BY period, instrument) AS volume,
            AVG(turn_base)    OVER (PARTITION BY period, instrument) AS turn_base,
            SUM(deal_number)  OVER (PARTITION BY period, instrument) AS deal_number,
            ROW_NUMBER()      OVER (PARTITION BY period, instrument ORDER BY date DESC) AS rn
        FROM data_orgn
    ),
    data2 AS (
        SELECT
            date,
            instrument,
            open,
            close,
            high,
            low,
            amount,
            volume,
            turn_base,
            deal_number,
        FROM data1
        WHERE rn = 1 
    )
    SELECT 
        date,
        instrument,
        open,
        close,
        high,
        low,
        amount,
        volume,
        volume / turn_base AS turn,
        deal_number,
    FROM data2
)
SELECT
    date,
    instrument,
    open,
    close,
    high,
    low,
    amount,
    volume,
    turn,
    deal_number,
FROM data_freq
ORDER BY date, instrument
"""

df_quarter = get_data_d(sql_quarter, sd, ed, freq="quarter", instruments=['600519.SH'])
df_quarter

,date,instrument,open,close,high,low,amount,volume,turn,deal_number
0,2020-01-02,600519.SH,8255.352493,8130.936720,8606.644089,7026.563767,3.000018e+11,277396329,0.220822,1590546
1,2020-04-01,600519.SH,8174.848169,10831.220435,10876.385072,8072.388121,2.135521e+11,162864247,0.129649,1028430
2,2020-07-01,600519.SH,10869.129114,12353.638915,13534.583120,10839.661039,4.142098e+11,246736746,0.196416,1553731
3,2020-10-09,600519.SH,12527.633829,14793.269734,14800.525693,11846.462250,3.369742e+11,190543620,0.151683,1188074
4,2021-01-04,600519.SH,14807.929731,14874.714162,19456.925761,14069.006649,5.853278e+11,274816165,0.218768,1818844
5,2021-04-01,600519.SH,14963.562629,15371.263822,17177.370262,13816.010639,4.247975e+11,203637352,0.162106,1416260
6,2021-07-01,600519.SH,15395.702988,13676.964455,15769.614754,11401.207254,5.521871e+11,317298148,0.252586,2236793
7,2021-10-08,600519.SH,13620.313422,15321.189690,16569.007168,12966.958103,4.218704e+11,217063137,0.172794,1525733
8,2022-01-04,600519.SH,15358.558445,12847.378086,15462.817272,11905.685452,3.867253e+11,213065561,0.169611,1553956
9,2022-04-01,600519.SH,12929.140923,15448.733306,15690.473876,12705.376816,3.426301e+11,187652515,0.149381,1392966


In [15]:
Kline(df_quarter)

## Quadmonth

In [16]:
sql_quadmonth = """
WITH 
data_orgn AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
        turn,
        volume / turn AS turn_base,
        deal_number,
    FROM cn_stock_bar1d
),
data_freq AS (
    WITH 
    data1 AS (
        SELECT
            instrument,
            CONCAT(CAST(year(date) AS CHAR), CAST(FLOOR((month(date)-1) / 4) AS CHAR)) AS period,
            FIRST_VALUE(date) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS date,
            FIRST_VALUE(open) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
            LAST_VALUE(close) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
            MAX(high)         OVER (PARTITION BY period, instrument) AS high,
            MIN(low)          OVER (PARTITION BY period, instrument) AS low,
            SUM(amount)       OVER (PARTITION BY period, instrument) AS amount,
            SUM(volume)       OVER (PARTITION BY period, instrument) AS volume,
            AVG(turn_base)    OVER (PARTITION BY period, instrument) AS turn_base,
            SUM(deal_number)  OVER (PARTITION BY period, instrument) AS deal_number,
            ROW_NUMBER()      OVER (PARTITION BY period, instrument ORDER BY date DESC) AS rn
        FROM data_orgn
    ),
    data2 AS (
        SELECT
            date,
            instrument,
            open,
            close,
            high,
            low,
            amount,
            volume,
            turn_base,
            deal_number,
        FROM data1
        WHERE rn = 1 
    )
    SELECT 
        date,
        instrument,
        open,
        close,
        high,
        low,
        amount,
        volume,
        volume / turn_base AS turn,
        deal_number,
    FROM data2
)
SELECT
    date,
    instrument,
    open,
    close,
    high,
    low,
    amount,
    volume,
    turn,
    deal_number,
FROM data_freq
ORDER BY date, instrument
"""

df_quadmonth = get_data_d(sql_quadmonth, sd, ed, freq="quadmonth", instruments=['600519.SH'])
df_quadmonth

,date,instrument,open,close,high,low,amount,volume,turn,deal_number
0,2020-01-02,600519.SH,8255.352493,9263.120258,9513.708263,7026.563767,3.741837e+11,339260157,0.270069,1967468
1,2020-05-06,600519.SH,9155.537207,13227.315506,13445.734653,9148.218632,4.408274e+11,281691913,0.224242,1772755
2,2020-09-01,600519.SH,13268.037720,14793.269734,14800.525693,11846.462250,4.497267e+11,256588872,0.204259,1620558
3,2021-01-04,600519.SH,14807.929731,14858.277196,19456.925761,14069.006649,7.338654e+11,347047373,0.276268,2317417
4,2021-05-06,600519.SH,14808.077812,11644.104164,17177.370262,11401.207254,6.685810e+11,354541339,0.282234,2477615
5,2021-09-01,600519.SH,11651.577915,15321.189690,16569.007168,11443.060260,5.817365e+11,311226090,0.247752,2202598
6,2022-01-04,600519.SH,15358.558445,13664.856978,15462.817272,11905.685452,5.023481e+11,278133186,0.221409,2031932
7,2022-05-05,600519.SH,13676.964455,14534.651775,15690.473876,12742.820309,4.192511e+11,222234445,0.176910,1661480
8,2022-09-01,600519.SH,14445.132221,13212.612477,14474.216633,10070.005622,5.179796e+11,318907324,0.253867,2361890
9,2023-01-03,600519.SH,13244.745061,13469.061099,14803.940442,12883.712529,3.405299e+11,189186793,0.150603,1458427


In [17]:
Kline(df_quadmonth)

## Semiannual

In [18]:
sql_semiannual = """
WITH 
data_orgn AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
        turn,
        volume / turn AS turn_base,
        deal_number,
    FROM cn_stock_bar1d
),
data_freq AS (
    WITH 
    data1 AS (
        SELECT
            instrument,
            CONCAT(CAST(year(date) AS CHAR), CAST(FLOOR((month(date)-1) / 6) AS CHAR)) AS period,
            FIRST_VALUE(date) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS date,
            FIRST_VALUE(open) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
            LAST_VALUE(close) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
            MAX(high)         OVER (PARTITION BY period, instrument) AS high,
            MIN(low)          OVER (PARTITION BY period, instrument) AS low,
            SUM(amount)       OVER (PARTITION BY period, instrument) AS amount,
            SUM(volume)       OVER (PARTITION BY period, instrument) AS volume,
            AVG(turn_base)    OVER (PARTITION BY period, instrument) AS turn_base,
            SUM(deal_number)  OVER (PARTITION BY period, instrument) AS deal_number,
            ROW_NUMBER()      OVER (PARTITION BY period, instrument ORDER BY date DESC) AS rn
        FROM data_orgn
    ),
    data2 AS (
        SELECT
            date,
            instrument,
            open,
            close,
            high,
            low,
            amount,
            volume,
            turn_base,
            deal_number,
        FROM data1
        WHERE rn = 1 
    )
    SELECT 
        date,
        instrument,
        open,
        close,
        high,
        low,
        amount,
        volume,
        volume / turn_base AS turn,
        deal_number,
    FROM data2
)
SELECT
    date,
    instrument,
    open,
    close,
    high,
    low,
    amount,
    volume,
    turn,
    deal_number,
FROM data_freq
ORDER BY date, instrument
"""

df_semiannual = get_data_d(sql_semiannual, sd, ed, freq="semiannual", instruments=['600519.SH'])
df_semiannual

,date,instrument,open,close,high,low,amount,volume,turn,deal_number
0,2020-01-02,600519.SH,8255.352493,10831.220435,10876.385072,7026.563767,5.135538e+11,440260576,0.350471,2618976
1,2020-07-01,600519.SH,10869.129114,14793.269734,14800.525693,10839.661039,7.511840e+11,437280366,0.348099,2741805
2,2021-01-04,600519.SH,14807.929731,15371.263822,19456.925761,13816.010639,1.010125e+12,478453517,0.380874,3235104
3,2021-07-01,600519.SH,15395.702988,15321.189690,16569.007168,11401.207254,9.740575e+11,534361285,0.425380,3762526
4,2022-01-04,600519.SH,15358.558445,15448.733306,15690.473876,11905.685452,7.293553e+11,400718076,0.318993,2946922
5,2022-07-01,600519.SH,15524.277234,13212.612477,15620.746830,10070.005622,7.102235e+11,418556879,0.333193,3108380
6,2023-01-03,600519.SH,13244.745061,13135.793036,14803.940442,12378.695418,4.968014e+11,280650105,0.223412,2164092
7,2023-07-03,600519.SH,13197.937533,13562.407261,15031.200192,12555.130393,4.978817e+11,282104491,0.224570,2169778
8,2024-01-02,600519.SH,13475.972452,11769.190812,13969.438115,11341.056911,5.829574e+11,353601586,0.281486,2798438
9,2024-07-01,600519.SH,11781.141346,12414.367713,15319.141095,9992.170445,6.961918e+11,460765515,0.366794,3543787


In [19]:
Kline(df_semiannual)

## Year

In [20]:
sql_year = """
WITH 
data_orgn AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
        turn,
        volume / turn AS turn_base,
        deal_number,
    FROM cn_stock_bar1d
),
data_freq AS (
    WITH 
    data1 AS (
        SELECT
            instrument,
            CAST(year(date) AS CHAR) AS period,
            FIRST_VALUE(date) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS date,
            FIRST_VALUE(open) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
            LAST_VALUE(close) OVER (PARTITION BY period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
            MAX(high)         OVER (PARTITION BY period, instrument) AS high,
            MIN(low)          OVER (PARTITION BY period, instrument) AS low,
            SUM(amount)       OVER (PARTITION BY period, instrument) AS amount,
            SUM(volume)       OVER (PARTITION BY period, instrument) AS volume,
            AVG(turn_base)    OVER (PARTITION BY period, instrument) AS turn_base,
            SUM(deal_number)  OVER (PARTITION BY period, instrument) AS deal_number,
            ROW_NUMBER()      OVER (PARTITION BY period, instrument ORDER BY date DESC) AS rn
        FROM data_orgn
    ),
    data2 AS (
        SELECT
            date,
            instrument,
            open,
            close,
            high,
            low,
            amount,
            volume,
            turn_base,
            deal_number,
        FROM data1
        WHERE rn = 1 
    )
    SELECT 
        date,
        instrument,
        open,
        close,
        high,
        low,
        amount,
        volume,
        volume / turn_base AS turn,
        deal_number,
    FROM data2
)
SELECT
    date,
    instrument,
    open,
    close,
    high,
    low,
    amount,
    volume,
    turn,
    deal_number,
FROM data_freq
ORDER BY date, instrument
"""

df_year = get_data_d(sql_year, sd, ed, freq="year", instruments=['600519.SH'])
df_year

,date,instrument,open,close,high,low,amount,volume,turn,deal_number
0,2020-01-02,600519.SH,8255.352493,14793.269734,14800.525693,7026.563767,1.264738e+12,877540942,0.698569,5360781
1,2021-01-04,600519.SH,14807.929731,15321.189690,19456.925761,11401.207254,1.984183e+12,1012814802,0.806254,6997630
2,2022-01-04,600519.SH,15358.558445,13212.612477,15690.473876,10070.005622,1.439579e+12,819274955,0.652186,6055302
3,2023-01-03,600519.SH,13244.745061,13562.407261,15031.200192,12378.695418,9.946831e+11,562754596,0.447982,4333870
4,2024-01-02,600519.SH,13475.972452,12414.367713,15319.141095,9992.170445,1.279149e+12,814367101,0.648279,6342225
5,2025-01-02,600519.SH,12414.367713,11633.028397,13505.838271,11404.356261,1.177663e+12,798826106,0.636580,6195804
6,2026-01-05,600519.SH,11699.083874,12546.317168,13244.883404,11167.007850,2.616401e+11,183665352,0.146666,1366356


In [21]:
Kline(df_year)